In [1]:
# Dependencies
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
#base_url = "https://pokeapi.co/api/v2/pokemon/"

In [2]:
# Call the data from the api
def get_info():
    starter_lst = []
    for i in range(150):
        url = "https://pokeapi.co/api/v2/pokemon/"+str(i+1)
        response = requests.get(url).json()
        location = requests.get(response["location_area_encounters"]).json()

        if len(location) ==0:
            location="Whitechapel"
        else:
            location = location[0]["location_area"]["name"]
            # Initialise the data for the dataframe     

        starter_lst.append({
        "Pokemon name": response["name"],
        "Weight": response["weight"],
        "Fighting type" : response["types"][0]["type"]["name"],
        "Height": response["height"],
        "Main ability": response["abilities"][0]["ability"]["name"],
        "Attack": response["stats"][1]["base_stat"],
        "Defense": response["stats"][2]["base_stat"],
        "Type of Move":response["moves"][0]["move"]["name"],
        "Region": location
        })

        # Print the initial dataframe

    final_df = pd.DataFrame(starter_lst) 
    return (final_df)
    

In [3]:
final_df = get_info()
final_df

,Pokemon name,Weight,Fighting type,Height,Main ability,Attack,Defense,Type of Move,Region
0,bulbasaur,69,grass,7,overgrow,49,49,razor-wind,cerulean-city-area
1,ivysaur,130,grass,10,overgrow,62,63,swords-dance,Whitechapel
2,venusaur,1000,grass,20,overgrow,82,83,swords-dance,Whitechapel
3,charmander,85,fire,6,blaze,52,43,mega-punch,pallet-town-area
4,charmeleon,190,fire,11,blaze,64,58,mega-punch,Whitechapel
...,...,...,...,...,...,...,...,...,...
145,moltres,600,fire,20,pressure,100,90,razor-wind,mt-silver-top
146,dratini,33,dragon,18,shed-skin,64,45,bind,mt-coronet-4f
147,dragonair,165,dragon,40,shed-skin,84,65,bind,mt-coronet-4f
148,dragonite,2100,dragon,22,inner-focus,134,95,mega-punch,dragonspiral-tower-outside


In [4]:
# Call the second api for the growth rate
import pprint

def get_growth_info ():
    base_url_2 = "https://pokeapi.co/api/v2/growth-rate/"

    lis_df=[]
    for i in range(1,7):
        url = "https://pokeapi.co/api/v2/growth-rate/"+str(i)+"/"
        response = requests.get(url).json()
        growth_lst = []
        
        for j in response["pokemon_species"]:
            growth_lst.append(j["name"])
            level_dic={"Growth Rate":[response["name"]]*len(growth_lst),"Pokemon name":growth_lst}
            
        lis_df.append(pd.DataFrame(level_dic)) 

    growth_df=pd.concat(lis_df)
    return (growth_df)



In [5]:
growth_df = get_growth_info()
growth_df

,Growth Rate,Pokemon name
0,slow,growlithe
1,slow,tentacool
2,slow,shellder
3,slow,exeggcute
4,slow,rhyhorn
...,...,...
9,fast-then-very-slow,hariyama
10,fast-then-very-slow,swalot
11,fast-then-very-slow,wailord
12,fast-then-very-slow,crawdaunt


In [6]:
# Merge the growth rate dataframe and the initial dataframe
merge_df = pd.merge(final_df, growth_df, on="Pokemon name")
merge_df


,Pokemon name,Weight,Fighting type,Height,Main ability,Attack,Defense,Type of Move,Region,Growth Rate
0,bulbasaur,69,grass,7,overgrow,49,49,razor-wind,cerulean-city-area,medium-slow
1,ivysaur,130,grass,10,overgrow,62,63,swords-dance,Whitechapel,medium-slow
2,venusaur,1000,grass,20,overgrow,82,83,swords-dance,Whitechapel,medium-slow
3,charmander,85,fire,6,blaze,52,43,mega-punch,pallet-town-area,medium-slow
4,charmeleon,190,fire,11,blaze,64,58,mega-punch,Whitechapel,medium-slow
...,...,...,...,...,...,...,...,...,...,...
145,moltres,600,fire,20,pressure,100,90,razor-wind,mt-silver-top,slow
146,dratini,33,dragon,18,shed-skin,64,45,bind,mt-coronet-4f,slow
147,dragonair,165,dragon,40,shed-skin,84,65,bind,mt-coronet-4f,slow
148,dragonite,2100,dragon,22,inner-focus,134,95,mega-punch,dragonspiral-tower-outside,slow


In [7]:
# Re-order the columns to tidy up ready for the merge with the co-ordinate data
reorder_df = merge_df.reindex(columns=['Pokemon name', 'Fighting type', 'Growth Rate', 'Main ability', 'Height', 'Weight', 'Attack', 'Defense', 'Type of Move', 'Region'])
reorder_df

,Pokemon name,Fighting type,Growth Rate,Main ability,Height,Weight,Attack,Defense,Type of Move,Region
0,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area
1,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel
2,venusaur,grass,medium-slow,overgrow,20,1000,82,83,swords-dance,Whitechapel
3,charmander,fire,medium-slow,blaze,6,85,52,43,mega-punch,pallet-town-area
4,charmeleon,fire,medium-slow,blaze,11,190,64,58,mega-punch,Whitechapel
...,...,...,...,...,...,...,...,...,...,...
145,moltres,fire,slow,pressure,20,600,100,90,razor-wind,mt-silver-top
146,dratini,dragon,slow,shed-skin,18,33,64,45,bind,mt-coronet-4f
147,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f
148,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside


In [10]:
# Export to CSV to insert latitude and longitude for locations, and map to real world locations for the interactive map,
# call back into code for analysis
location_path = "location_with_lat_long.csv"

location_data = pd.read_csv(location_path)
location_data


,Pokemon name,Latitude,Longitude
0,bulbasaur,6.8428,81.3399
1,ivysaur,51.5148,0.0651
2,venusaur,51.5148,0.0651
3,charmander,7.2936,80.6413
4,charmeleon,51.5148,0.0651
...,...,...,...
145,moltres,50.4155,-15.0737
146,dratini,39.9042,116.4074
147,dragonair,39.9042,116.4074
148,dragonite,50.3755,-4.1427


In [12]:
# Merge the dataframe with latitude and longitude to final_df
merged_df = pd.merge(reorder_df, location_data, on="Pokemon name", how ='left')
merged_df

,Pokemon name,Fighting type,Growth Rate,Main ability,Height,Weight,Attack,Defense,Type of Move,Region,Latitude,Longitude
0,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area,6.8428,81.3399
1,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel,51.5148,0.0651
2,venusaur,grass,medium-slow,overgrow,20,1000,82,83,swords-dance,Whitechapel,51.5148,0.0651
3,charmander,fire,medium-slow,blaze,6,85,52,43,mega-punch,pallet-town-area,7.2936,80.6413
4,charmeleon,fire,medium-slow,blaze,11,190,64,58,mega-punch,Whitechapel,51.5148,0.0651
...,...,...,...,...,...,...,...,...,...,...,...,...
145,moltres,fire,slow,pressure,20,600,100,90,razor-wind,mt-silver-top,50.4155,-15.0737
146,dratini,dragon,slow,shed-skin,18,33,64,45,bind,mt-coronet-4f,39.9042,116.4074
147,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f,39.9042,116.4074
148,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside,50.3755,-4.1427


In [13]:
# Define the API endpoints for male, female, and genderless Pokemon
male_url = "https://pokeapi.co/api/v2/gender/1/"
female_url = "https://pokeapi.co/api/v2/gender/2/"
genderless_url = "https://pokeapi.co/api/v2/gender/3/"

In [14]:
# Define a function to get the name and gender of each Pokemon from a given API endpoint
def get_pokemon_gender(api_url, gender):
    response = requests.get(api_url)
    if response.status_code == 200:
        pokemon_list = response.json()["pokemon_species_details"]
        pokemon_names = [pokemon["pokemon_species"]["name"] for pokemon in pokemon_list]
        pokemon_genders = [response.json()["name"]]*len(pokemon_list)
        return pd.DataFrame({"Name": pokemon_names, "Gender": pokemon_genders})
    else:
        print(f"Error fetching data from {api_url}")

In [15]:
# Call the function for each API endpoint to get the male, female, and genderless Pokemon
male_df = get_pokemon_gender(male_url, "male")
female_df = get_pokemon_gender(female_url, "female")
genderless_df = get_pokemon_gender(genderless_url, "genderless")

In [16]:
# Concatenate the three dataframes into a single dataframe
pokemon_df = pd.concat([male_df, female_df, genderless_df])

In [18]:
# Print the resulting dataframe

pokemon1_df = pokemon_df.rename(columns={'Name': 'Pokemon name'})
pokemon1_df


,Pokemon name,Gender
0,bulbasaur,female
1,charmander,female
2,squirtle,female
3,caterpie,female
4,weedle,female
...,...,...
141,lunala,genderless
142,naganadel,genderless
143,polteageist,genderless
144,maushold,genderless


In [20]:
# Rename female and male to genderless to avoid duplications in the data
pokemon1_df['Gender'] = pokemon1_df['Gender'].str.replace('female', 'genderless')
pokemon1_df['Gender'] = pokemon1_df['Gender'].str.replace('male', 'genderless')
second_merge_df = pd.merge(merged_df, pokemon1_df, on="Pokemon name", how ='left')
second_merge_df

,Pokemon name,Fighting type,Growth Rate,Main ability,Height,Weight,Attack,Defense,Type of Move,Region,Latitude,Longitude,Gender
0,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area,6.8428,81.3399,genderless
1,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area,6.8428,81.3399,genderless
2,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel,51.5148,0.0651,genderless
3,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel,51.5148,0.0651,genderless
4,venusaur,grass,medium-slow,overgrow,20,1000,82,83,swords-dance,Whitechapel,51.5148,0.0651,genderless
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f,39.9042,116.4074,genderless
272,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f,39.9042,116.4074,genderless
273,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside,50.3755,-4.1427,genderless
274,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside,50.3755,-4.1427,genderless


In [21]:
# Drop duplicates where Pokemon had genders, renumber index and keep change permanently
second_merge_df.drop_duplicates( keep='first', inplace=True, ignore_index=True)
second_merge_df

,Pokemon name,Fighting type,Growth Rate,Main ability,Height,Weight,Attack,Defense,Type of Move,Region,Latitude,Longitude,Gender
0,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area,6.8428,81.3399,genderless
1,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel,51.5148,0.0651,genderless
2,venusaur,grass,medium-slow,overgrow,20,1000,82,83,swords-dance,Whitechapel,51.5148,0.0651,genderless
3,charmander,fire,medium-slow,blaze,6,85,52,43,mega-punch,pallet-town-area,7.2936,80.6413,genderless
4,charmeleon,fire,medium-slow,blaze,11,190,64,58,mega-punch,Whitechapel,51.5148,0.0651,genderless
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,moltres,fire,slow,pressure,20,600,100,90,razor-wind,mt-silver-top,50.4155,-15.0737,genderless
146,dratini,dragon,slow,shed-skin,18,33,64,45,bind,mt-coronet-4f,39.9042,116.4074,genderless
147,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f,39.9042,116.4074,genderless
148,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside,50.3755,-4.1427,genderless


In [22]:
from sqlalchemy import create_engine

In [23]:
engine = create_engine("sqlite:///pokemon.sqlite")

In [24]:
second_merge_df.to_sql("pokemon", con=engine, if_exists='replace')

150

In [26]:
df = pd.read_sql("select * from pokemon", con=engine)
df

,index,Pokemon name,Fighting type,Growth Rate,Main ability,Height,Weight,Attack,Defense,Type of Move,Region,Latitude,Longitude,Gender
0,0,bulbasaur,grass,medium-slow,overgrow,7,69,49,49,razor-wind,cerulean-city-area,6.8428,81.3399,genderless
1,1,ivysaur,grass,medium-slow,overgrow,10,130,62,63,swords-dance,Whitechapel,51.5148,0.0651,genderless
2,2,venusaur,grass,medium-slow,overgrow,20,1000,82,83,swords-dance,Whitechapel,51.5148,0.0651,genderless
3,3,charmander,fire,medium-slow,blaze,6,85,52,43,mega-punch,pallet-town-area,7.2936,80.6413,genderless
4,4,charmeleon,fire,medium-slow,blaze,11,190,64,58,mega-punch,Whitechapel,51.5148,0.0651,genderless
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,moltres,fire,slow,pressure,20,600,100,90,razor-wind,mt-silver-top,50.4155,-15.0737,genderless
146,146,dratini,dragon,slow,shed-skin,18,33,64,45,bind,mt-coronet-4f,39.9042,116.4074,genderless
147,147,dragonair,dragon,slow,shed-skin,40,165,84,65,bind,mt-coronet-4f,39.9042,116.4074,genderless
148,148,dragonite,dragon,slow,inner-focus,22,2100,134,95,mega-punch,dragonspiral-tower-outside,50.3755,-4.1427,genderless
